# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [20]:
#!pip install selenium
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Chrome()
# pegar cotação dólar
navegador.get('https://www.google.com.br/')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
                      ).send_keys('cotação dólar')  # acha o campo de busca por onde a tag dele está no html, coisa
# que muda muita menos do que o layout, informação essa que usavamos com o pyautogui

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
                      ).send_keys(Keys.ENTER)

dolar = float(navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                      ).get_attribute('data-value'))

print(dolar)

# pegar cotação euro
navegador.get('https://www.google.com.br/')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
                      ).send_keys('cotação euro')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
                      ).send_keys(Keys.ENTER)

euro = float(navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                      ).get_attribute('data-value'))

print(euro)

# pegar cotação ouro
navegador.get('https://dolarhoje.com/ouro-hoje/')  # não usei o mesmo site que eles, porque esse é um site que já conhecia, então resolvi tentar com ele

ouro = float(navegador.find_element(By.XPATH, '//*[@id="nacional"]').get_attribute('value').replace(',', '.'))

print(ouro)

5.534369
6.342
325.93


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [27]:
import pandas as pd

dados = pd.read_excel('Produtos.xlsx')
# display(dados.info())
display(dados)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [24]:
dados.loc[dados['Moeda'] == 'Dólar', 'Cotação'] = dolar
dados.loc[dados['Moeda'] == 'Euro', 'Cotação'] = euro
dados.loc[dados['Moeda'] == 'Ouro', 'Cotação'] = ouro

dados['Preço de Compra'] = dados['Preço Original'] * dados['Cotação']
dados['Preço de Venda'] = dados['Preço de Compra'] * dados['Margem']

display(dados)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.534369,5534.313656,1.40,7748.039119
1,Carro Renault,4500.00,Euro,6.342000,28539.000000,2.00,57078.000000
2,Notebook Dell,899.99,Dólar,5.534369,4980.876756,1.70,8467.490486
3,IPhone,799.00,Dólar,5.534369,4421.960831,1.70,7517.333413
4,Carro Fiat,3000.00,Euro,6.342000,19026.000000,1.90,36149.400000
5,Celular Xiaomi,480.48,Dólar,5.534369,2659.153617,2.00,5318.307234
6,Joia 20g,20.00,Ouro,325.930000,6518.600000,1.15,7496.390000


### Agora vamos exportar a nova base de preços atualizada

In [25]:
dados.to_excel('Novos Produtos.xlsx', index=False)